In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import TextLoader, PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from kiwipiepy import Kiwi

# reorder
from langchain_community.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda

# multi-query
from langchain.retrievers.multi_query import MultiQueryRetriever

# reranker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

kiwi = Kiwi()

In [4]:
# kiwi 함수
def kiwi_tokenize(docs):
    return [token.form for token in kiwi.tokenize(docs)]

In [5]:
loaders = [
    PDFPlumberLoader("data/Adaptive_RAG.pdf"),
    PDFPlumberLoader("data/Naive_RAG.pdf"),
    PDFPlumberLoader('data/RAPTOR_RAG.pdf'),
    PDFPlumberLoader('data/Self_RAG.pdf')
]

docs = []

for loader in loaders:
    docs.extend(loader.load())

In [6]:
len(docs)

87

In [7]:
# Ensemble Retriever (kiwi bm25 + chroma similarity)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

documents = text_splitter.split_documents(docs)

chroma_vector_store = Chroma.from_documents(documents, embedding)
chroma_retriever = chroma_vector_store.as_retriever()

kiwi_vector_store = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, kiwi_vector_store],
    weights=[0.5, 0.5],
    search_kwargs={"k": 10}
    #search_type="mmr"
)

In [8]:
# MultiQUery Retriever

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = ensemble_retriever,
    llm=llm
)

In [10]:
# Reranker

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개 모델 선택
compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor, base_retriever = ensemble_retriever
)

c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# Reorder
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs

In [12]:
# prompt
template = """
    당신은 주어진 문서에 대해서 QA를 해주는 assistant bot입니다.
    주어진 문서를 이용해서 답변해 주세요
    만약 주어진 질문에 대해서 모른다면, 모른다고 답변해 주세요
    문서의 출처와 페이지 넘버를 작성해주세요
    한국어로 답변해주세요

    #Example Format:
    (brief summary of the answer)
    (table)
    (detailed answer to the question)

    **출처**
    - (page source and page number)

    # Question:
    {question}

    # Context:
    {context}

    # Answer
"""
prompt = PromptTemplate.from_template(template, input_variable=["question", "context"])

In [13]:
# chain
chain = ({"context":ensemble_retriever| RunnableLambda(reorder_documents), "question": RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser()
         )

In [14]:
print(chain.invoke("AdaptiveRAG에 대해서 설명해 주세요"))

Adaptive-RAG는 적응형 검색 기반 생성 모델로, 복잡한 질문에 대해 효과적으로 대응하기 위해 설계되었습니다. 이 모델은 질문의 복잡성을 분류하는 오라클 분류기를 사용하여, 단순한 질문은 LLM(대형 언어 모델)만으로 처리하고, 복잡한 질문은 검색기와 LLM을 함께 활용하여 해결합니다. Adaptive-RAG는 성능과 효율성 모두에서 우수한 결과를 보여줍니다.

| Adaptive-RAG의 특징 | 설명 |
|---------------------|------|
| 질문 처리 방식     | 단순 질문은 LLM, 복잡한 질문은 검색기와 LLM 결합 |
| 성능                | 기존 모델보다 더 효과적이고 효율적 |
| 구현 세부사항       | A100 GPU 사용, PyTorch 및 Transformers 라이브러리 활용 |

Adaptive-RAG는 복잡한 질문에 대한 적응형 접근 방식을 통해, 다양한 질문 유형에 대해 최적의 성능을 발휘하도록 설계되었습니다. 이 모델은 질문의 복잡성을 기반으로 적절한 처리 전략을 선택하여, 효율적인 정보 검색과 생성 과정을 가능하게 합니다.

**출처**
- Adaptive_RAG.pdf, 페이지 6, 7, 13


In [15]:
print(chain.invoke("Self-RAG에 대해서 설명해 주세요"))

Self-RAG(자기 반영적 검색 증강 생성)는 대형 언어 모델(LLM)의 품질과 사실성을 향상시키기 위해 설계된 프레임워크입니다. 이 프레임워크는 검색과 자기 반영을 통해 LLM이 생성한 텍스트의 질을 높이고, 필요할 경우 검색된 구문을 기반으로 텍스트를 생성하며, 생성된 결과를 비판적으로 평가할 수 있도록 학습합니다. Self-RAG는 일반적인 RAG 접근 방식과 달리, 검색된 자료의 완전한 지원을 보장하지 않고 무작위로 구문을 검색하는 대신, 필요에 따라 검색을 수행하고 결과의 관련성을 확인하는 과정을 포함합니다.

| Self-RAG의 주요 특징 |
|---------------------|
| 검색과 자기 반영을 통한 품질 향상 |
| 필요에 따라 검색된 구문을 기반으로 텍스트 생성 |
| 생성된 결과에 대한 비판적 평가 |
| 사용자 선호에 맞춘 모델 동작 조정 가능 |

Self-RAG는 LLM이 자신의 생성 과정을 반영하고, 특정 작업 입력에 대해 작업 출력을 생성하는 동시에 중간에 특별한 토큰을 생성하여 검색의 필요성을 신호합니다. 이러한 방식은 LLM의 창의성과 다재다능성을 해치지 않으면서도 사실적 정확성을 높이는 데 기여합니다.

**출처**
- Self-RAG.pdf, 페이지 2, 0, 15
